## Compute Causal Effects from Data According to Graph: Run Cell Below

In [ ]:
from Graphs import CausalGraph, CausalNode, CausalEdge, GroupedCausalGraph, GroupedCausalNode, GroupedCausalEdge
import pandas as pd
import numpy as np
from causal_inference import compute_causal_effect

grouped_graph = GroupedCausalGraph()

# Create Grouped Nodes
grouped_graph.add_node('1&2')
grouped_graph.add_node('3')
grouped_graph.add_node('4')
grouped_graph.add_node('static',dynamic=False)

# Add Edges
grouped_graph.add_edge('static','1&2')
grouped_graph.add_edge('static','3')
grouped_graph.add_edge('static','4')

grouped_graph.add_edge('1&2','3',time_to_effect={'min':1,'max':2})
grouped_graph.add_edge('1&2','4',time_to_effect={'min':1,'max':3})
grouped_graph.add_edge('3','4',time_to_effect={'min':2,'max':2})

# Create Causal Graphs
grouped_graph.getNode('1&2').graph.add_node('X1')
grouped_graph.getNode('1&2').graph.add_node('X2')
grouped_graph.getNode('3').graph.add_node('X3')
grouped_graph.getNode('4').graph.add_node('X4')
grouped_graph.getNode('static').graph.add_node('age')
grouped_graph.getNode('static').graph.add_node('sex')


# Load Data
data = pd.read_csv('simple_data.csv')

# Compute Causal Effects
causal_result = compute_causal_effect(data, causal_graph=grouped_graph,
                              cause_variable='X3',response_variable='X4',
                              delta_t_values=np.arange(1,10),
                              intervention_values=np.linspace(-1,2,6))
print(causal_result)

## More Causal Graph Tests Below (Unrelated)

In [7]:
grouped_graph = GroupedCausalGraph()
grouped_graph.add_node(node="GroupA")
grouped_graph.add_node(node="GroupB")
grouped_graph.add_node(node="GroupC")
grouped_graph.add_edge(from_node="GroupA",to_node="GroupB", time_to_effect={'min':1,'max':1})
grouped_graph.add_edge(from_node="GroupC",to_node="GroupB", time_to_effect={'min':2,'max':3})
grouped_graph.add_edge(from_node="GroupA",to_node="GroupC", time_to_effect={'min':3,'max':4})

print(grouped_graph.getIncomingEdges(grouped_graph.getNode("GroupB")))
print(grouped_graph.getOutgoingEdges(grouped_graph.getNode("GroupB")))

print(grouped_graph.getParents(grouped_graph.getNode("GroupA")))
print(grouped_graph.getParents(grouped_graph.getNode("GroupB")))
print(grouped_graph.getParents(grouped_graph.getNode("GroupC")))

[GroupA --[1,1]--> GroupB    (dynamic -> dynamic), GroupC --[2,3]--> GroupB    (dynamic -> dynamic)]
[]
[]
[GroupA, GroupC]
[GroupA]


In [8]:
groupA = CausalGraph(parent=grouped_graph.getNode("GroupA"))
groupA.add_node("cat")
groupA.add_node("dog")
groupA.add_node("parrot")
groupA.add_edge("cat","dog", time_to_effect={'min':1,'max':2})

groupB = CausalGraph(parent=grouped_graph.getNode("GroupB"))
groupB.add_node("zebra")
groupB.add_node("fish")
groupB.add_edge("zebra","fish", time_to_effect={'min':2,'max':2})

groupC = CausalGraph(parent=grouped_graph.getNode("GroupC"))
groupC.add_node("horse")

grouped_graph.getNode("GroupA").graph = groupA
grouped_graph.getNode("GroupB").graph = groupB
grouped_graph.getNode("GroupC").graph = groupC

print(grouped_graph.getParents(groupA.getNode("dog")))
print(grouped_graph.getParents(groupA.getNode("cat")))
print(grouped_graph.getParents(groupB.getNode("fish")))
print(grouped_graph.getParents(groupC.getNode("horse")))

[cat (dynamic)]
[]
[zebra (dynamic), cat (dynamic), dog (dynamic), parrot (dynamic), horse (dynamic)]
[cat (dynamic), dog (dynamic), parrot (dynamic)]


In [10]:
print(grouped_graph.getStaticDynamicNodes())
print(grouped_graph.getTemporalCopiesOfParent(groupC.getNode("horse"),groupB.getNode("fish")))
print(grouped_graph.getFlattenedNode('zebra'))

([], [cat (dynamic), dog (dynamic), parrot (dynamic), zebra (dynamic), fish (dynamic), horse (dynamic)])
['horse_tm2', 'horse_tm3']
zebra (dynamic)
